# Katas

Welcome to some katas

The exercises are designed to make you familiar with the following key concept in Polars:

* Eager vs lazy
* Context and expressions

Let's get started!

In [ ]:
import polars as pl

print(pl.__version__)

## Kata 1: Eager Mode

Here is some code to list a bunch of URLs about the ever-famous NYC Yellow Taxi trip data.

In [ ]:
base = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-{:02d}.parquet"

# we keep the files from Feb 2023 to retain those with the same schema
urls = tuple(base.format(month) for month in range(2, 10))

* Read the first parquet file in the list using `pl.read_parquet`.
* Display the top five rows.

How long does this take?

## Kata 2: Lazy mode

Repeat the exercise above, using `pl.scan_parquet` instead. What happens if you run the code? What changes if you scan the whole set of URLs?

Write the code to materialize the result.

## Kata 3: The schema

Display the `LazyFrame` schema.

How is this different from the `pandas` schema?

Like `pandas`, `Polars` can `describe` the dataset. Can you do that on a `LazyFrame`?

## Kata 4: A peek at Expressions

Expressions are the "domain specific language" (DSL) of Polars. They are a powerful abstraction to describe complex operations over data with an elegant and consistent syntax.

Expressions start with a column selector, like `pl.col()` or `pl.all()`. Expressions describe a transformation and are not evaluated on their own. Observe what happens after running this statement:

In [ ]:
pl.col("VendorID")

* What is the type of this object?
* What methods are available with method chaining? Use the `dir()` builtin function to list some of them.

Expressions can describe operations on multiple columns, like this:

In [ ]:
pl.col("VendorID", "tpep_pickup_datetime", "tpep_dropoff_datetime")

Expressions branch from this small core, and use method chaining. For example:

* `pl.col("a").max()`
* `pl.col("date").dt.year()`
* `pl.col("name").str.to_uppercase()`

Where `dt` and `str` are examples of `namespaces`, i.e. group of operation that belong to the same data type. There are also namespaces lists, categoricals and struct types - but enough for the moment: now we need to understand how to materialise the results of these operations. On with contexts!

## Kata 4: The first context: `pl.select`

The `select()` context is used to select columns. Now that we know the existence of the column selector, do the following exercises:

In [ ]:
# data = pl.scan_parquet(urls[0])
# suggestion: you might want to persist this data to disk to avoid being blocked because of sending too many requests
# data.sink_parquet("../data/trips.parquet")

data = pl.scan_parquet("../data/trips.parquet")

1. Select all columns. *Hint: you don't have to write out all names, nor access the `columns` attribute of the dataframe. You might want to take a look at the `pl.all()` expression in the docs...*
3. Select all columns except `VendorID`. *Hint: once again, you might want to have a look at `pl.all().exclude()`...*
4. Select all columns that contain `amount` in their name. *Hint: you can use regex patterns.*
5. Select all `Int64` columns. *Hint: Polars has different datatypes that you can use inside `pl.col`.*
6. Select all `Int64` and `Int32` columns.
6. Select all datetime and string columns, minus the first column.

> **Hint**. To inspect the intermediate steps or results of a query, you can always call the `fetch()` method. It is like a debug statement.

## Kata 5: The hidden technique of Column Selectors

Selecting columns in this way is arguably handy already. However, Polars has a `selectors` module we can use to write even more complex selections.

Refer to the [docs](https://docs.pola.rs/py-polars/html/reference/selectors.html#selectors) and repeat the previous kata, plus the following:

7. Select all columns that are integers or datetime, except the first one.
8. Select all columns that contain an "ID" or "amount" and are not floating point numbers.

In [ ]:
import polars.selectors as cs

## Kata 6: Building up expressions

It's time to explore expressions more thoroughly. As a starter, we can describe mathematical operations on Expressions like with similar Python objects, such as `numpy` arrays or `pandas` Series:

In [ ]:
data.select(pl.col("trip_distance") * 1000).fetch(5)

This is useful, but breaks the method chain if we want to use other operations. An alternative is to wrap the expression in parenthesis.

1. Multiply the `trip_distance` by 1000 to transform it in metres and name it `trip_distance_meters`.
2. Count the number of rows in the data. *Hint: you can use `pl.count()` inside the context.*
4. Add `tolls_amount` to `Airport_fee` and name it `total_fees`.
5. Compute the average trip distance.
6. Select the smallest and largest `tpep_pickup_datetime`. *Hint: this will likely raise an error the first time you run it: remember, there can't be columns with duplicate names in the DataFrame. Note: you don't have to call `pl.col(...).min`: you can just use `pl.max()`*.
7. Count the different unique values of `passenger_count`.
8. Get the number of unique values of `VendorID` and `RatecodeID`.

> **Hint**. You can call `.alias` on an expression to rename the column it generates. Similarly, you can access the `.name.suffix` method to add a suffix. Alternatively, you can name the column using a kwarg notation (i.e., `col=pl.some.expr`).

## Kata 7: The query plan

The `LazyFrame` represents a *Logical Plan*, i.e. a sequence of transformations. It embodies a query, rather than a `DataFrame`. You can inspect this plan when you print the `repr` of the `LazyFrame`.

* What method does it suggest to call, to inspect the optimized plan?
* Inspect the plan of the last two exercises in the previous kata, comparing the optimised and unoptimised queries.
* If you have `graphviz` on your `$PATH`, do the same with `data.show_graph`.

## Kata 8: the second context: `with_columns`

Try to write the expressions in the sixth kata in the same `with_column` context. Do you notice any errors popping up? Can you explain them?

Call `explain` on both and compare the query plans.

## Kata 9: Data types

You can change the memory representation of a numeric datatype with `.cast()`.

1. Cast the string column into a categorical.
2. Cast the integer columns to have the smallest memory footprint.
3. Cast the datetime columns to milliseconds.

## Kata 10: Namespaces

Polars segregates operations on similar data types behind namespaces, e.g. `str`, `dt`, `list` and `struct`. These are pretty powerful and much more versatile than their `pandas` counterparts, and allow chaining operations without losing performance.

1. Cast the `store_and_fwd_flag` column to lowercase.
2. Extract the year, month and day of the temporal columns.
3. Perform the following steps. *Hint: this can be written as a singular expression. You can always call `.fetch()` to materialise the intermediate results.*
    1. Cast the temporal columns to strings.
    1. Split them at the ` ` (space) mark
    2. Take the first element
    3. Split the element at the `-` mark.
    4. Cast the result into a struct.
    5. Cast the struct into a JSON string with.

## Kata 11: The third context: `filter`

Filtering, combined with the query optimiser, can translate in huge gains. Thanks to the so-called "predicate pushdown", query engines can scan parquet files to just read the required rows - thus saving bandwidth and I/O.

Filtering is done inside the `filter` context and uses basic Python logical operators. Perform the following filtering operations:

1. Passenger count is greater than 3.
2. The dropoff hour is the same as the pickup's.
2. Trip distance is greater than the average trip distance. *Note: this is the biggest difference between Polars DSL and regular SQL, where such expressions would require separate statements.*

## Kata 12: The fourth context: `group_by`

Aggregations with `group_by` can be elegantly expressed in Polars. An aggregation looks like this: `data.group_by(...).agg(...)`. Inside the brackets, there can be any expression!

1. Aggregate by passenger count and compute the average and standard deviation of the trip distance. *Hint: remember, there can't be two columns with the same name!*
2. Aggregate by month of departure and compute the mean and standard deviation of the total amount and trip distance.
4. Aggregate by vendor ID and just write `"passenger_count` inside the `.agg` context. What happens?

## Kata 14: Window Functions

Window functions can be powerful allies to generate features. This are especially hard with pandas, since they would require creating a new dataframe and performing a join.

Window functions are just computed as this: `pl.col(...).mean().over(...)`. They are especially useful for time-based data. For those cases, you can use the powerful `Expr.rolling()` to perform rolling window computations across datetime columns.

1. Compute the mean and standard deviation of the price over vendor ID and passenger count.
2. Compute the rolling window of the price over a week. Use the pickup time as index. *Note: these operations require you to sort the index, or mark the index as sorted using `set_sorted`.

## Kata 15: Manipulating the elements of a list

Aggregate the data by vendor and passenger count on trip distance and fare amount. Compute the rolling mean of size 3 over the elements of the list.

# Solutions

## Kata 1: Eager mode

In [ ]:
import polars as pl

url = urls[0]

pl.read_parquet(url).head()

## Kata 2: Lazy mode

In [ ]:
pl.scan_parquet(url).head().collect()

pl.scan_parquet(urls).head()

## Kata 3: The schema

In [ ]:
data = pl.scan_parquet(url)

data.schema

## Kata 4: Selecting columns

In [ ]:
# 1. Select all columns
data.select(pl.all())
data.select("*")

# 2. Select all columns except `VendorID`.
data.select(pl.all().exclude("VendorID"))

# 3. Select all columns that contain `amount` in their name
data.select(pl.col(r"^*amount$"))

# 4. Select all integer columns.
data.select(pl.col(pl.Int64))

# 5. Select all numeric columns.
data.select(pl.col(pl.NUMERIC_DTYPES))

# 6. Select all datetime and string columns.
data.select(pl.col(pl.Datetime), pl.col(pl.Utf8))

## Kata 5: Columns selectors

In [ ]:
# 1. Select all columns
data.select(cs.all())

# 2. Select all columns except `VendorID`.
data.select(~cs.by_name("VendorID"))

# 3. Select all columns that contain `amount` in their name
data.select(cs.contains("amount"))
data.select(cs.matches("*amount"))

# 4. Select all integer columns.
data.select(cs.integer())

# 5. Select all numeric columns.
data.select(cs.numeric())

# 6. Select all datetime and string columns.
data.select(cs.temporal(), cs.string())
data.select(cs.temporal() | cs.string())

# 7. Select all columns that are integers or datetime, except the first one.
data.select(cs.integer() - cs.first() | cs.temporal())

# 8. Select all columns that contain an "ID" or "amount" and are not floating point numbers.
data.select(cs.contains(("ID", "Amount")) | ~cs.float())

## Kata 6: Introduction to expressions

In [ ]:
# 1. Multiply the `trip_distance` by 1000 to cast it in metres.
data.with_columns(trip_distance_meters=pl.col("trip_distance") * 1000)
data.with_columns(pl.col("trip_distance").mul(1000).alias("trip_distance_meters"))
data.with_columns(pl.col("trip_distance").mul(1000).name.suffix("_meters"))

# 2. Add `tolls_amount`, `Airport_fee` and name it `total_fees`.
data.with_columns(total_fees=pl.col("tolls_amount") + pl.col("Airport_fee"))

#
data.select(
    pl.min("tpep_pickup_datetime").name.suffix("_min"),
    pl.max("tpep_pickup_datetime").name.suffix("_max"),
).collect()

# 3. Compute the ratio between `tip`, `mta_tax` and `fare_amount` over `total_amount`.
data.with_columns(
    pl.col("tip", "mta_tax", "fare_amount").truediv("total_amount").name.suffix("_pct")
)

# 4. Compute the average trip distance.
data.with_columns(pl.col("trip_distance").mean())

# 5. Count the unique values of `passenger_count`.
data.with_columns(pl.col("passenger_count").value_counts())

# 6. Get the unique values of `VendorID` and `RatecodeID`.
data.with_columns(pl.col("VendorID", "RatecodeID").n_unique())

## Kata 7: The query plan

In [ ]:
percentage_change = (
    pl.col("tip_amount", "mta_tax", "fare_amount")
    .truediv("total_amount")
    .name.suffix("_pct")
)

data.with_columns(percentage_change).explain(optimized=False)
data.with_columns(percentage_change).explain(optimized=True)

## Kata 8: Chaining multiple contexts 

In [ ]:
data.with_columns(
    pl.col("trip_distance").mul(1000).name.suffix("_meters"),
    pl.col("tolls_amount").add(pl.col("Airport_fee")).alias("total_fees"),
).with_columns(
    pl.col("tip_amount", "total_fees", "mta_tax", "fare_amount")
    .truediv(pl.col("total_amount"))
    .name.suffix("_pct")
).explain()

## Kata 9: Data types

In [ ]:
data.select(
    cs.temporal().as_expr().cast(pl.Date),
    cs.string().as_expr().cast(pl.Categorical),
    cs.numeric().shrink_dtype(),
).fetch()

## Kata 10: Namespaces

In [ ]:
data.select(
    cs.temporal()
    .as_expr()
    .cast(pl.Utf8)
    .str.split(" ")
    .list.first()
    .str.split("-")
    .list.to_struct()
    .struct.json_encode()
).fetch()

## Kata 11: The third context: `filter`

In [ ]:
data.filter(pl.count("passenger_count") > 3).fetch()

data.filter(pl.col("tpep_pickup_datetime") == pl.col("tpep_pickup_datetime")).fetch()

(data.lazy().filter(pl.col("trip_distance") > pl.col("trip_distance").mean()).fetch())

## Kata 12: The fourth context: `group_by`

In [ ]:
(
    data.group_by("passenger_count")
    .agg(
        pl.col("trip_distance").mean().name.suffix("_mean"),
        pl.col("trip_distance").std().name.suffix("_std"),
    )
    .fetch()
)

(
    data.group_by(pl.col("tpep_pickup_datetime").dt.month())
    .agg(
        pl.col("total_amount", "trip_distance").mean().name.suffix("_mean"),
        pl.col("total_amount", "trip_distance").std().name.suffix("_std"),
    )
    .fetch()
)

(data.group_by("VendorID").agg("passenger_count").fetch())

## Kata 14: Window functions

In [ ]:
(
    data.with_columns(
        pl.mean("total_amount")
        .over("VendorID", "passenger_count")
        .name.suffix("_over_vendorid_and_passenger_count")
    ).fetch()
)

(
    data.set_sorted("tpep_pickup_datetime")
    .with_columns(
        pl.mean("total_amount")
        .rolling("tpep_pickup_datetime", period="1w")
        .name.suffix("_rolling")
    )
    .fetch()
)

## Kata 15: List manipulation

In [ ]:
(
    data.group_by("VendorID", "passenger_count")
    .agg("trip_distance", "fare_amount")
    .with_columns(
        pl.col("trip_distance").list.eval(pl.element().rolling_mean(window_size=3))
    )
    .fetch()
)